In [140]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
from impyute.imputation.cs import fast_knn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import re
from copy import deepcopy

In [141]:
columns_to_drop = [
    'Unnamed: 0',
    'Планировка',
    'Строительная серия',
    'Сдан',
    'Газоснабжение',
    'Аварийность',
    'Лифты',
    'Срок сдачи',
    'Отделка',
    'Подъезды',
    'id',
    'Тип перекрытий',
    'Мусоропровод',
    'Построен',
    'Ремонт',
    'Отопление'
]
df = pd.read_excel(r"C:\Users\ilyak\Downloads\RawFlatsData.xlsx").drop(columns_to_drop, axis=1)
indexes_of_rows_to_drop = []
for i, row in df.iterrows():
    total_na = row.isnull().sum()
    if total_na > 5 or type(row.loc['price']) != str or float(row.loc['price'].replace(' ', '').replace('₽', '')) >= 300_000_000:
        indexes_of_rows_to_drop.append(i)
df = df.drop(index=indexes_of_rows_to_drop, axis=0)
df = df.reset_index(drop=True)
df.shape

(1064, 14)

In [142]:
def count_bathrooms(s):
    result = 0
    for symbol in s:
        if symbol.isdigit():
            result += int(symbol)
    return result

In [143]:
"""
Куча страшно выглядящей предобработки
"""
df['Площадь'] = df['Общая'].apply(lambda s: float(s[:-3].replace(',', '.')) if type(s) == str else s)
df['price'] = df['price'].apply(lambda s: float(s[:-2].replace(',', '.').replace(' ', '')) if type(s) == str else s)
df['Новойстройка'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Новостройка' in s else 0))
df['Пентхаус'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Пентхаус' in s else 0))
df['Апартаменты'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Апартаменты' in s else 0))
df['Кухня'] = df['Кухня'].apply(lambda s: s if type(s) != str else float(s[:-3].replace(',', '.')))
df['Вид на улицу'] = df['Вид из окон'].apply(lambda s: s if type(s) != str else (1 if 'улицу' in s else 0))
df['Парковка'] = df['Парковка'].apply(lambda s: 'Наземная' if (type(s) != str or s == 'Открытая') else s)
df['Санузел'] = df['Санузел'].apply(lambda s: count_bathrooms(s) if type(s) == str else None)
df['Высота потолков'] = df['Высота потолков'].apply(lambda s: float(s[:-2].replace(',', '.')) if type(s) == str else s)
df['Жилплощадь'] = df['Жилая'].apply(lambda s: float(s[:-3].replace(',', '.')) if type(s) == str else s)
df['Балкон'] = df['Балкон/лоджия'].apply(lambda s: sum([(int(symb) if symb.isdigit() else 0) for symb in s]) if type(s) == str else 0)
df['Время до метро'] = df['sub'].apply(lambda s: int(re.search(r'\d{1,2} мин.', s)[0][:-5]) if (type(s) == str and 'мин.' in s) else None)
df['Всего этажей'] = df['Этаж'].apply(lambda s: s if type(s) != str else int(s.split()[-1]))
df['Этаж'] = df['Этаж'].apply(lambda s: s if type(s) != str else int(s.split()[0]))
df = df.drop(['Тип жилья', 'Вид из окон', 'Балкон/лоджия', 'sub', 'Общая', 'Жилая'], axis=1)

In [144]:
"""
Здесь я перевожу качественные переменные в количественные.
Но т.к. я в дальнейшем хочу NaNы заполнить определенными знчениями, то я их не буду
кодировать числами, а просто оставлю NaNами.
Поэтому я применяю кодировку к столбцам с качественными переменными, не кодируя NaNы.
"""
house_type_encoding = {
    None: None,
    'Кирпичный': 0,
    'Блочный': 1,
    'Монолитный': 2,
    'Монолитно кирпичный': 3,
    'Сталинский': 4,
    'Панельный': 5
}
parking_encoding = {
    None: None,
    'Подземная': 0,
    'Открытая': 1,
    'Наземная': 2,
    'Многоуровневая': 3
}


df['Тип дома'] = df['Тип дома'].apply(lambda s: house_type_encoding[s] if type(s) == str else s)
df['Парковка'] = df['Парковка'].apply(lambda s: parking_encoding[s] if type(s) == str else s)
df.head()

,Кухня,price,Год постройки,Санузел,Тип дома,Парковка,Высота потолков,Этаж,Площадь,Новойстройка,Пентхаус,Апартаменты,Вид на улицу,Жилплощадь,Балкон,Время до метро,Всего этажей
0,23.1,37821977.0,NaN,NaN,NaN,2,3.0,4,105.3,1,0,0,0.0,44.8,0,9.0,9
1,NaN,139000000.0,2016.0,3.0,2.0,2,4.5,4,200.0,0,0,1,1.0,NaN,0,6.0,6
2,21.0,29900000.0,2012.0,1.0,2.0,2,2.8,2,78.0,0,0,0,1.0,38.0,1,2.0,10
3,8.7,24800000.0,2021.0,2.0,5.0,2,2.7,3,75.7,0,0,0,1.0,51.4,0,19.0,14
4,18.6,50574792.0,NaN,NaN,NaN,2,3.0,6,95.5,1,0,0,1.0,42.2,0,7.0,20


In [145]:
"""
А теперь заменяем NaNы наиболее вероятными значениями.
Их мы находим с помощью метода ближайших соседей (то есть NaN будет заполнен тем же значением,
которое чаще стоит у ближайших соседей)
"""
columns = list(df.columns)
sys.setrecursionlimit(100000)
imputer = KNNImputer(n_neighbors=5)
imputer.fit(df.dropna())
df = pd.DataFrame(imputer.fit_transform(df))
df.columns = columns
for column in columns:
    if column not in ['Кухня', 'price', 'Высота потолков', 'Площадь', 'Жилплощадь']:
        df[column] = df[column].apply(round)
df

,Кухня,price,Год постройки,Санузел,Тип дома,Парковка,Высота потолков,Этаж,Площадь,Новойстройка,Пентхаус,Апартаменты,Вид на улицу,Жилплощадь,Балкон,Время до метро,Всего этажей
0,23.1,37821977.0,1993,2,2,2,3.00,4,105.3,1,0,0,0,44.80,0,9,9
1,20.0,139000000.0,2016,3,2,2,4.50,4,200.0,0,0,1,1,141.80,0,6,6
2,21.0,29900000.0,2012,1,2,2,2.80,2,78.0,0,0,0,1,38.00,1,2,10
3,8.7,24800000.0,2021,2,5,2,2.70,3,75.7,0,0,0,1,51.40,0,19,14
4,18.6,50574792.0,1972,2,1,2,3.00,6,95.5,1,0,0,1,42.20,0,7,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,26.0,23367161.0,2019,2,2,2,3.05,19,114.9,1,0,1,0,67.40,1,18,24
1060,16.2,18300000.0,2021,1,2,0,2.95,11,50.1,0,0,0,1,31.30,1,5,20
1061,13.5,20914657.0,1984,2,2,2,2.95,4,67.2,1,0,0,1,41.00,0,6,32
1062,16.2,20378500.0,2011,1,2,2,3.00,14,76.9,1,0,1,0,36.66,2,11,21


In [146]:
df.to_excel(r"C:\Users\ilyak\Downloads\processed_data.xlsx", index=False)

In [147]:
# ТаДааааа